In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## What is RAG (retrieval augmented generation)?
Basically, shoving lot of extra information in the prompt.

In [ ]:
# Example 

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(
    prompt_template.format(
        context="Ajit has two sisters, Preeti and Sweta. Ajit is male.",
        question="Who is Preeti's bother?"         
    ))

print(response)

content="Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 119, 'total_tokens': 152, 'completion_time': 0.124850027, 'prompt_time': 0.01691466, 'queue_time': 0.04664075, 'total_time': 0.141764687}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--71f47147-2fdb-4da1-9d80-617a67ce05f6-0' usage_metadata={'input_tokens': 119, 'output_tokens': 33, 'total_tokens': 152}


In [ ]:
print(response.content)

Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti.


>RAG is all about cleverly pushing is as much information in the context with minimum possible tokens

In [ ]:
# little non-trival RAG example

# grab a novel
file = open("indianTales.txt",encoding="utf-8")
text = file.read()

In [ ]:
# lets try to talk with this book

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Prepare your prompt
prompt_template = """You are a novel reader. You are given collection of stories:
{collection_of_stories}
You are tasked to make a list of story titles in this collection. Write a short summary for each story in Hindi Language. Skip the story from the list, if the story is not provided in the text. 
"""

response = model.invoke(prompt_template.format(collection_of_stories = text[:len(text)//20]))

print(response)

content='यहाँ दी गई कहानियों की सूची है:\n\n1. **शेर और बगुला** - इस कहानी में, एक बगुला एक शेर को उसके गले में फंसी हुई हड्डी से मुक्त करता है, लेकिन शेर उसका शुक्रिया नहीं अदा करता है और उसे बताता है कि वह एक शिकारी है और उसे अपने शिकार के लिए हमेशा तैयार रहना होता है।\n\n2. **राजकुमार और राजकुमारी लबाम** - इस कहानी में, एक राजकुमार अपनी माँ की सलाह के विरुद्ध जाकर एक जंगल में जाता है और वहाँ एक पक्षी राजा से मिलता है, जो उसे राजकुमारी लबाम के बारे में बताता है। राजकुमार उसकी खोज में निकल पड़ता है और कई चुनौतियों का सामना करता है।' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 268, 'prompt_tokens': 5232, 'total_tokens': 5500, 'completion_time': 0.912887512, 'prompt_time': 0.397401711, 'queue_time': 0.050460227, 'total_time': 1.310289223}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--039ca613-33ce-4912-a9ef-25da3ad3838d-0' usage_metadata={'

In [ ]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

यहाँ दी गई कहानियों की सूची है:

1. **शेर और बगुला** - इस कहानी में, एक बगुला एक शेर को उसके गले में फंसी हुई हड्डी से मुक्त करता है, लेकिन शेर उसका शुक्रिया नहीं अदा करता है और उसे बताता है कि वह एक शिकारी है और उसे अपने शिकार के लिए हमेशा तैयार रहना होता है।

2. **राजकुमार और राजकुमारी लबाम** - इस कहानी में, एक राजकुमार अपनी माँ की सलाह के विरुद्ध जाकर एक जंगल में जाता है और वहाँ एक पक्षी राजा से मिलता है, जो उसे राजकुमारी लबाम के बारे में बताता है। राजकुमार उसकी खोज में निकल पड़ता है और कई चुनौतियों का सामना करता है।

### Problem: Too much raw information in the context makes the prompt too long.
- Costly
- adds noise
### Solution: Use RAG
https://python.langchain.com/docs/tutorials/rag/
### To understand RAG, we need to understand Semantic Search
https://python.langchain.com/docs/tutorials/retrievers/

In [ ]:
# load text using RELEVANT loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt",encoding="utf-8")
docs = loader.load()

In [ ]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

Split given book into 563 sub-documents.


### Embedding

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


GoogleGenerativeAIError: Error embedding content: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

In [ ]:
document_ids

['87a9d873-c0d6-4241-b518-56690ad20c0e',
 '4cea4a6f-ba1d-46ee-bffa-73dc3043bad5',
 '0d347631-bb68-4ebb-8bd0-8eeaa9bd4fdb',
 'ac238ec6-5573-41de-ba85-383fcb204cb5',
 'c9958570-42fb-4876-a795-5a0e683a9d63',
 '851d96cc-abeb-44b9-9fdd-4355757bbaaf',
 '8a26ca51-404a-4503-8b67-6b13634bed1f',
 'bcdb94b3-4383-4460-a335-af494cd630fe',
 '385db14b-cac7-46da-9dcd-503478f543df',
 'ac3351de-423a-4c74-aaaf-be05d81ad3d3',
 'b8984a09-72ee-4df6-a955-2bfa38ac72cc',
 '69736dfd-f397-4055-93e5-6e54f8203c79',
 'bd5ce582-56fd-41f6-a3e7-3c6e03deab57',
 '77a72c26-0f37-4882-85ab-c31d9e564941',
 'a6bf06a8-eb33-4143-82ee-258fd144596d',
 '1c562824-9eab-41ec-ab31-3717a996101b',
 '97c0c52a-6ffe-4481-bd2a-9cfb12117558',
 '761ed0c5-1619-4432-8a63-ac6f4caeab69',
 '3a7435e7-637d-4f6c-bf90-88cd109c0d62',
 '86873eca-d5b4-46b8-99a7-b694bd49cf1e',
 '9c02d3b5-a5e9-4533-a90b-317f6bf652f6',
 '9b33f84e-2f07-4603-a607-8449c38bef88',
 '7d3dbe5c-2f7c-441c-9f4b-7600cfeb42ca',
 'a83078ae-7f63-46f8-b2b9-1db558f3b659',
 'd42d736d-49f8-

In [ ]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

In [ ]:
search_results

[(Document(id='568caa1e-cb9e-4e92-9c71-d43d98ae40b3', metadata={'source': 'indianTales.txt', 'start_index': 117080}, page_content='How came the crown in the jaws of the tiger? The king of Ujjaini had a\nweek before gone with all his hunters on a hunting expedition. All of\na sudden the tiger-king started from the wood, seized the king, and\nvanished.'),
  0.7410933853822097),
 (Document(id='058356b1-c334-42cc-92cb-990c4860fb0c', metadata={'source': 'indianTales.txt', 'start_index': 110013}, page_content='goldsmith. Do not release him; and if you do, you shall surely repent\nof it one day or other." Thus advising, the hungry tiger went away\nwithout waiting for an answer.'),
  0.7157341176789158),
 (Document(id='c65dac5b-16ed-475e-991a-da38db5a383e', metadata={'source': 'indianTales.txt', 'start_index': 110182}, page_content='[Illustration:]'),
  0.7022634590788741),
 (Document(id='86873eca-d5b4-46b8-99a7-b694bd49cf1e', metadata={'source': 'indianTales.txt', 'start_index': 12770}, page_

### What is RAG?
Retrieve using semantic search and dump the similar chunks in the context of the prompt.
LLM sees the question and retrieved docs in its prompt and generates tokens accordingly.

In [ ]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

In [ ]:
doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

How came the crown in the jaws of the tiger? The king of Ujjaini had a
week before gone with all his hunters on a hunting expedition. All of
a sudden the tiger-king started from the wood, seized the king, and
vanished.


goldsmith. Do not release him; and if you do, you shall surely repent
of it one day or other." Thus advising, the hungry tiger went away
without waiting for an answer.


[Illustration:]


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the lion's mouth, striking the stick with his beak so that
it fell out, and then settled on a branch. The lion gets well, and
one day was eating a buffalo he had killed. The crane thinking "I will
sound him," settled on a branch just over him, and in conversation
spoke this first verse:


"Where do you come from? Who are you?" asked the king, entering the
room.

"O king!" replied the prince, "I am the son of a king who rules over
such-and-su

In [ ]:
# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)



from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))

In [ ]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

There is no mention of a lion playing a role in the story provided, except in a separate incident where a crane helps a lion by removing a bone from its mouth. The lion is not related to the main story about the king and the tiger. I don't know the role of the lion in the overall story context.

# RAG Summary

In [ ]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt",encoding="utf-8")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


Split given book into 563 sub-documents.


In [ ]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

How came the crown in the jaws of the tiger? The king of Ujjaini had a
week before gone with all his hunters on a hunting expedition. All of
a sudden the tiger-king started from the wood, seized the king, and
vanished.


goldsmith. Do not release him; and if you do, you shall surely repent
of it one day or other." Thus advising, the hungry tiger went away
without waiting for an answer.


[Illustration:]


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the lion's mouth, striking the stick with his beak so that
it fell out, and then settled on a branch. The lion gets well, and
one day was eating a buffalo he had killed. The crane thinking "I will
sound him," settled on a branch just over him, and in conversation
spoke this first verse:


"Where do you come from? Who are you?" asked the king, entering the
room.

"O king!" replied the prince, "I am the son of a king who rules over
such-and-su

In [ ]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))


from IPython.display import Markdown
Markdown(response.content)

The lion is not mentioned in the provided context as playing a role in the main story, but rather in a separate anecdote where a crane helps a lion by removing a bone from its mouth. The lion is portrayed as a character in this separate story, but its role is not connected to the main narrative. I don't know the lion's role in the overall story.